In [1]:
import numpy as np

In [43]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [44]:
def std(X):
    X = (X - X.mean()) / X.std()
    return X

In [45]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [46]:
def eval_model(X, y, iters, alpha):
    w = [10, 0.1, 0.1, 0.1]
    n = X.shape[1]
    for i in range(iters+1):
        z = np.dot(X.T, w)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot(X, y_pred - y))
        #if i % (iters / 10) == 0:
        #    print(w, err)
    print(w, err)
    return w
        

In [47]:
X[1] = std(X[1])
X[2] = std(X[2])
X[3] = std(X[3])

### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log. 

В np.log попадет 0, если y_pred = 0, либо (1 - y_pred) = 0. <br>
Значит, нам нужно вместо 0 и 1 подставить близкие к ним значения

In [48]:
def calc_logloss(y, y_pred):
    y_pred[y_pred==0] = 1e-10
    y_pred[y_pred==1] = 1 - 1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [49]:
alpha = 0.01
iters = np.linspace(100, 50000, 10, dtype='int')
for i in iters:
    eval_model(X, y, i, alpha)

[9.49506434 0.09998068 0.15725936 0.29395418] 4.685142810324271
[ 0.2136248  -2.22085167 -0.84680521  3.63869427] 0.3417912423249548
[ 0.43226933 -2.83106151 -1.32378965  4.85253252] 0.3028954728148881
[ 0.63049249 -3.20746511 -1.73374446  5.74696185] 0.2820858703765397
[ 0.80999224 -3.498882   -2.09731878  6.50018691] 0.26733195071040294
[ 0.97529966 -3.7513906  -2.4263351   7.17312841] 0.25556104468113494
[ 1.12930513 -3.98200499 -2.72879383  7.79224179] 0.24560533531011872
[ 1.27402824 -4.19806695 -3.01042006  8.37154987] 0.2369002270299374
[ 1.4107797  -4.40298803 -3.27498984  8.91885041] 0.2291380031711391
[ 1.54060298 -4.5987523  -3.52532209  9.43944067] 0.2221237773086437


как видим, существенное увеличение количества итераций значительное снижает log loss.

### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba)

In [50]:
def calc_pred_proba(w, X):
    z = np.dot(X.T, w)
    prob = sigmoid(z)
    return(prob)

ранее мы уже выполняли эти операции внутри функии eval_model

In [51]:
w = eval_model(X, y, 5000, 0.01)

[ 0.19223912 -2.11605281 -0.7841385   3.45515045] 0.3494146096966278


In [52]:
calc_pred_proba(w, X)

array([0.28350111, 0.22259881, 0.91187061, 0.25183055, 0.61039477,
       0.34151171, 0.96375432, 0.21303764, 0.30020866, 0.87351604])

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [53]:
def calc_pred(w, X):
    y_pred = calc_pred_proba(w, X)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    return y_pred

In [54]:
calc_pred(w, X), y

(array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [55]:
y_pred = calc_pred(w, X)

In [56]:
def accuracy(y, y_pred): # доля правильных ответов от выборки
    res = 0
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            res +=1
    return res/len(y)

In [57]:
accuracy(y, y_pred)

0.9

In [58]:
def conf_matrix(y, y_pred):
    matrix = [[0,0], [0,0]]
    for i in range(len(y)):
        if y_pred[i] == 1 and y_pred[i] == y[i]:
            matrix[0][0] += 1
        if y_pred[i] == 0 and y_pred[i] == y[i]:
            matrix[1][1] += 1  
        if y_pred[i] == 1 and y_pred[i] != y[i]:
            matrix[0][1] += 1
        if y_pred[i] == 0  and y_pred[i] != y[i]:
            matrix[1][0] += 1
    return matrix

In [59]:
y_pred, y

(array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

In [60]:
conf_matrix(y, y_pred)

[[4, 0], [1, 5]]

In [61]:
def precision(y, y_pred):
    matrix = conf_matrix(y,y_pred)
    tp = matrix[0][0]
    tn = matrix[1][1]
    fn = matrix[1][0]
    fp = matrix[0][1]
    prec = tp / (tp + fp)
    return prec

In [62]:
precision(y, y_pred)

1.0

In [63]:
def recall(y, y_pred):
    matrix = conf_matrix(y,y_pred)
    tp = matrix[0][0]
    tn = matrix[1][1]
    fn = matrix[1][0]
    fp = matrix[0][1]
    rec = tp / (tp + fn)
    return rec

In [64]:
recall(y, y_pred)

0.8

In [65]:
def f1_score(y, y_pred):
    rec = recall(y, y_pred)
    prec = precision(y, y_pred)
    F = 2*prec*rec / (prec + rec)
    return F    

In [66]:
f1_score(y, y_pred)

0.888888888888889

### 6. Могла ли модель переобучиться? Почему?

модель могла переобучиться, в первую очередь из-за слишком маленькой выборки.

### 7.  Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [67]:
def eval_model_lasso(X, y, iters, alpha, lambda_):
    n = X.shape[1]
    np.random.seed(10)
    w = np.random.randn(X.shape[0])
    #w = np.array([10, 0.1, 0.1, 0.1])
    for i in range(iters+1):
        y_pred = calc_pred_proba(w, X)
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot(X, y_pred - y) + lambda_ * w)
    print(w, err)
    return w   

In [68]:
y_pred_l1 = calc_pred(eval_model_lasso(X, y, 5000, 0.01, 0.01), X)

[ 0.09418186 -1.38365689 -0.78799358  2.64905661] 0.3962288392485325


In [69]:
y_pred_l1, y

(array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

In [70]:
def eval_model_ridge(X, y, iters, alpha, lambda_):
    n = X.shape[1]
    np.random.seed(10)
    w = np.random.randn(X.shape[0])
    #w = np.array([10, 0.1, 0.1, 0.1])
    for i in range(iters+1):
        y_pred = calc_pred_proba(w, X)
        err = calc_logloss(y, y_pred)
        w -= alpha * (2/n * np.dot(X, y_pred - y) + lambda_ * np.sign(w))
    print(w, err)
    return w   

In [71]:
y_pred_l2 = calc_pred(eval_model_ridge(X, y, 5000, 0.01, 0.01), X)

[ 0.26935407 -2.37056453 -1.11943642  4.12063162] 0.3250700298940703


In [72]:
y_pred_l2, y

(array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))